In [ ]:
from tensorflow.keras.models import load_model
import cv2
import tensorflow as tf
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import cv2

data_wd = Path('/data') / 'raspi_face_detection'
img_path = data_wd / 'images'
labels_path = data_wd / 'labels'

model_sets_path = data_wd / 'model_sets'
aug_sets_path = data_wd / 'aug_data'

In [ ]:
facetracker = load_model('facetracker.h5')

#### 11.2.1 Single Photo

In [ ]:
final_test_wd = data_wd / 'final_test_images'
img_gen = final_test_wd.glob('*.jpg')

In [ ]:
frame_captured = cv2.imread(str(next(img_gen)))
# cap = cv2.VideoCapture(0)
# _, frame_captured = cap.read()

frame = frame_captured[30:500, 50:500,:]

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
resized = tf.image.resize(frame, (120,120))

frame_size = frame.shape[0:2]

yhat = facetracker.predict(np.expand_dims(resized/255,0))
labels, sample_coords = [x[0] for x in yhat]
labels, sample_coords

if yhat[0][0][0] > 0.7:
    # Controls the main rectangle
    cv2.rectangle(frame, 
                  tuple(np.multiply(sample_coords[0:2], frame_size).astype(int)),
                  tuple(np.multiply(sample_coords[2:4], frame_size).astype(int)), 
                        (0,255,0), 2)
    # Controls the label rectangle
    cv2.rectangle(frame, 
                  tuple(np.add(np.multiply(sample_coords[0:2], frame_size).astype(int), 
                                [0,-30])),
                  tuple(np.add(np.multiply(sample_coords[0:2], frame_size).astype(int),
                                [130,0])), 
                        (0,255,0), -1)
    # Controls the text rendered
    cv2.putText(frame, 'brendan', tuple(np.add(np.multiply(sample_coords[0:2], frame_size).astype(int),
                                           [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
if yhat[0][0][1] > 0.9:
    # Controls the main rectangle
    cv2.rectangle(frame, 
                  tuple(np.multiply(sample_coords[4:6], frame_size).astype(int)),
                  tuple(np.multiply(sample_coords[6:8], frame_size).astype(int)), 
                        (255,0,0), 2)
    # Controls the label rectangle
    cv2.rectangle(frame, 
                  tuple(np.add(np.multiply(sample_coords[4:6], frame_size).astype(int), 
                                [0,-30])),
                  tuple(np.add(np.multiply(sample_coords[4:6], frame_size).astype(int),
                                [70,0])), 
                        (255,0,0), -1)
    
    # Controls the text rendered
    cv2.putText(frame, 'kara', tuple(np.add(np.multiply(sample_coords[4:6], frame_size).astype(int),
                                           [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

im = plt.imshow(frame)
im.axes.set_title(f'Brendan: {labels[0]:,.0%}      Kara: {labels[1]:,.0%}')
plt.show()

### 11.3 Real Time Detection

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[30:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]

    sample_coords_pixels = (sample_coords * (frame.shape[0:2] * 4)).round().astype(int)
    
    if yhat[0][0][0] > 0.75:
        # Controls the main rectangle
        cv2.rectangle(frame,                      
                      sample_coords_pixels[0:2], sample_coords_pixels[2:4], 
                      (0,0,255), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                       sample_coords_pixels[0:2] + [0,-30], sample_coords_pixels[0:2] + [130,0],
                      (0,0,255), -1)
        # Controls the text rendered
        cv2.putText(frame, 'brendan', sample_coords_pixels[0:2] + (0,-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    if yhat[0][0][1] > 0.75:
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      sample_coords_pixels[4:6], sample_coords_pixels[6:8], 
                      (0,255,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      sample_coords_pixels[4:6] + [0,-30], sample_coords_pixels[4:6] + [70,0], 
                      (0,255,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'kara', sample_coords_pixels[4:6] + [0,-5],
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()